# Predictor Final

## Carrega de dades i llibreries

In [165]:
#Exactament igual que a l'aproximació 1

# No volem gaires warnings
import warnings 
warnings.filterwarnings("ignore")

# Importem panda, numpy i seaborn
import pandas as pd
pd.options.display.max_rows = 999

import numpy as np
import seaborn as sb
sb.set(style="white", color_codes=True)

from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib

import glob

## Funcio prepare()

In [166]:
#Carrega les dades d'entrenament igual que aprox 1 però encapsulat en una funció. 
#A més exporta el dataFrame resultant.
def prepare():
    
    columns = [ 'id',
                'lliga',
                'temp',
                'temp_id',
                'date',
                'hour',
                'round',
                'result',
                'localId',
                'local_goals',
                'LALGS',
                'LALGR',
                'local_wins',
                'local_losses',
                'local_draws',
                'local_ga',
                'local_gf',
                'local_avg',
                'local_form',
                'local_mark',
                'local_points',
                'visitorId',
                'visitor_goals',
                'VAVGS',
                'VAVGR',
                'visitor_wins',
                'visitor_losses',
                'visitor_draws',
                'visitor_ga',
                'visitor_gf',
                'visitor_avg',
                'visitor_form',
                'visitor_mark',
                'visitor_points'
    ]

    full_data_set = pd.DataFrame(columns=columns)
    
    # Carreguem el fitxer de l'aproximació 1 (en aquest cas no fa falta concatenarlo amb cap més)
    for fname in glob.glob("../TF_data/1-aprox1.csv"):
        print ('Processing', fname)
        df = pd.read_csv(fname, 
                         sep=";", 
                         header=1,
                         names= columns,
                         na_values=['?'])

        df.set_index(keys = 'id', inplace = True)
        full_data_set = pd.concat([full_data_set, df],axis=0)
    # Establim l'index    
        full_data_set['id'] = full_data_set.index
    # I les dates com a datetime    
        full_data_set['date'] = pd.to_datetime(full_data_set.date)
    print('Finished')
    print 'Total', full_data_set.shape[0], 'partits'
    
    #Afegim els gols marcats i rebuts de les ultimes jornades
    full_data_set['local_goals_r_1'] = full_data_set['LALGR'].str.split('-').str[0].convert_objects(convert_numeric=True)
    full_data_set['local_goals_r_2'] = full_data_set['LALGR'].str.split('-').str[1].convert_objects(convert_numeric=True)
    full_data_set['local_goals_r_3'] = full_data_set['LALGR'].str.split('-').str[2].convert_objects(convert_numeric=True)
    full_data_set['local_goals_r_4'] = full_data_set['LALGR'].str.split('-').str[3].convert_objects(convert_numeric=True)

    full_data_set['local_goals_s_1'] = full_data_set['LALGS'].str.split('-').str[0].convert_objects(convert_numeric=True)
    full_data_set['local_goals_s_2'] = full_data_set['LALGS'].str.split('-').str[1].convert_objects(convert_numeric=True)
    full_data_set['local_goals_s_3'] = full_data_set['LALGS'].str.split('-').str[2].convert_objects(convert_numeric=True)
    full_data_set['local_goals_s_4'] = full_data_set['LALGS'].str.split('-').str[3].convert_objects(convert_numeric=True)

    full_data_set['visitor_goals_r_1'] = full_data_set['VAVGR'].str.split('-').str[0].convert_objects(convert_numeric=True)
    full_data_set['visitor_goals_r_2'] = full_data_set['VAVGR'].str.split('-').str[1].convert_objects(convert_numeric=True)
    full_data_set['visitor_goals_r_3'] = full_data_set['VAVGR'].str.split('-').str[2].convert_objects(convert_numeric=True)
    full_data_set['visitor_goals_r_4'] = full_data_set['VAVGR'].str.split('-').str[3].convert_objects(convert_numeric=True)

    full_data_set['visitor_goals_s_1'] = full_data_set['VAVGS'].str.split('-').str[0].convert_objects(convert_numeric=True)
    full_data_set['visitor_goals_s_2'] = full_data_set['VAVGS'].str.split('-').str[1].convert_objects(convert_numeric=True)
    full_data_set['visitor_goals_s_3'] = full_data_set['VAVGS'].str.split('-').str[2].convert_objects(convert_numeric=True)
    full_data_set['visitor_goals_s_4'] = full_data_set['VAVGS'].str.split('-').str[3].convert_objects(convert_numeric=True)
    
    # També afegim les victories o derrotes de les jornades anteriors

    #full_data_set['local_result_0'] = full_data_set['local_form'].str[0]
    full_data_set['local_result_1'] = full_data_set['local_form'].str[1]
    full_data_set['local_result_2'] = full_data_set['local_form'].str[2]
    full_data_set['local_result_3'] = full_data_set['local_form'].str[3]
    full_data_set['local_result_4'] = full_data_set['local_form'].str[4]

    #full_data_set['local_result_0'] = full_data_set['local_form'].str[0]
    full_data_set['visitor_result_1'] = full_data_set['visitor_form'].str[1]
    full_data_set['visitor_result_2'] = full_data_set['visitor_form'].str[2]
    full_data_set['visitor_result_3'] = full_data_set['visitor_form'].str[3]
    full_data_set['visitor_result_4'] = full_data_set['visitor_form'].str[4]
    
    # Les transformem en numerals (igual que result) 
    full_data_set = full_data_set.replace('l',0)
    full_data_set = full_data_set.replace('d',1)
    full_data_set = full_data_set.replace('w',2)
    
    #Eliminem les dades que no podrem reproduir
    full_data_set = full_data_set.drop('id', 1)
    full_data_set = full_data_set.drop('date', 1)
    full_data_set = full_data_set.drop('local_form', 1)
    full_data_set = full_data_set.drop('visitor_form', 1)
    full_data_set = full_data_set.drop('local_mark', 1)
    full_data_set = full_data_set.drop('visitor_mark', 1)
    full_data_set = full_data_set.drop('visitor_goals', 1)
    full_data_set = full_data_set.drop('local_goals', 1)

    full_data_set = full_data_set.drop('LALGR', 1)
    full_data_set = full_data_set.drop('LALGS', 1)
    full_data_set = full_data_set.drop('VAVGR', 1)
    full_data_set = full_data_set.drop('VAVGS', 1)
    
    # Els reemplaçem per -1 (només son de resultats de jornades anteriors. Per exemple quan no n'hi ha cap!, jornada1)
    full_data_set = full_data_set.fillna(0)
    
    joblib.dump(full_data_set, '../TF_data/full_data_set.pkl')

#Executem prepare (només per aquest exemple, normalment farem la crida desde fora!)
prepare()

('Processing', '../TF_data/1-aprox1.csv')
Finished
Total 2510 partits


## Entrenament del model

In [167]:
#Exactament que aprox1 però en una funció, a mes a mes carreguem el dataset desde el fitxer on l'hem exportat
#abans i exportem el classificador entrenat resultant.
def train() :
    full_data_set = joblib.load('../TF_data/full_data_set.pkl')
    
    # Definim la columna objectiu (que volem predir)
    y_name = 'result'
    feature_names = full_data_set.columns.tolist()
    feature_names.remove(y_name)

    # Separem el data set entre totes les dades, i els resultats
    X = full_data_set[feature_names]
    y = full_data_set[y_name]
    
    #Creem el model escollit amb els parametres optims decidits a l'aprox 1
    decision_tree_classifier = DecisionTreeClassifier(max_features= 45, max_depth= 4)

    #Entrenem el model
    decision_tree_classifier.fit(X,y)

    #Exportem el model ja entrenat
    joblib.dump(decision_tree_classifier, '../TF_data/predictor_model.pkl')

#Executem l'entrenamen
train()

## Preparació de la predicció 

In [168]:
#Per a predir un resultat, les es dades d'entrada s'han de deixar exactament igual que les del dataFrame, 
#per fer això utilitzem aquesta funció, que fa quasi els mateixos pasos que prepare() peró per la predicció
def prepare_x(x_file) :

    columns = [ 'id',
                'lliga',
                'temp',
                'temp_id',
                'date',
                'hour',
                'round',
                'localId',
                'local_goals',
                'LALGS',
                'LALGR',
                'local_wins',
                'local_losses',
                'local_draws',
                'local_ga',
                'local_gf',
                'local_avg',
                'local_form',
                'local_mark',
                'local_points',
                'visitorId',
                'visitor_goals',
                'VAVGS',
                'VAVGR',
                'visitor_wins',
                'visitor_losses',
                'visitor_draws',
                'visitor_ga',
                'visitor_gf',
                'visitor_avg',
                'visitor_form',
                'visitor_mark',
                'visitor_points'
    ]
    x = pd.DataFrame(columns=columns)

    for fname in glob.glob(x_file):
        df = pd.read_csv(fname, 
                         sep=";", 
                         header=1,
                         names= columns,
                         na_values=['?'])

        df.set_index(keys = 'id', inplace = True)
        x = pd.concat([x, df],axis=0)
        # Establim l'index    
        x['id'] = x.index
        # I les dates com a datetime    
        x['date'] = pd.to_datetime(x.date)

    x['local_goals_r_1'] = x['LALGR'].str.split('-').str[0].convert_objects(convert_numeric=True)
    x['local_goals_r_2'] = x['LALGR'].str.split('-').str[1].convert_objects(convert_numeric=True)
    x['local_goals_r_3'] = x['LALGR'].str.split('-').str[2].convert_objects(convert_numeric=True)
    x['local_goals_r_4'] = x['LALGR'].str.split('-').str[3].convert_objects(convert_numeric=True)

    x['local_goals_s_1'] = x['LALGS'].str.split('-').str[0].convert_objects(convert_numeric=True)
    x['local_goals_s_2'] = x['LALGS'].str.split('-').str[1].convert_objects(convert_numeric=True)
    x['local_goals_s_3'] = x['LALGS'].str.split('-').str[2].convert_objects(convert_numeric=True)
    x['local_goals_s_4'] = x['LALGS'].str.split('-').str[3].convert_objects(convert_numeric=True)

    x['visitor_goals_r_1'] = x['VAVGR'].str.split('-').str[0].convert_objects(convert_numeric=True)
    x['visitor_goals_r_2'] = x['VAVGR'].str.split('-').str[1].convert_objects(convert_numeric=True)
    x['visitor_goals_r_3'] = x['VAVGR'].str.split('-').str[2].convert_objects(convert_numeric=True)
    x['visitor_goals_r_4'] = x['VAVGR'].str.split('-').str[3].convert_objects(convert_numeric=True)

    x['visitor_goals_s_1'] = x['VAVGS'].str.split('-').str[0].convert_objects(convert_numeric=True)
    x['visitor_goals_s_2'] = x['VAVGS'].str.split('-').str[1].convert_objects(convert_numeric=True)
    x['visitor_goals_s_3'] = x['VAVGS'].str.split('-').str[2].convert_objects(convert_numeric=True)
    x['visitor_goals_s_4'] = x['VAVGS'].str.split('-').str[3].convert_objects(convert_numeric=True)

    # També afegim les victories o derrotes de les jornades anteriors

    #x['local_result_0'] = x['local_form'].str[0]
    x['local_result_1'] = x['local_form'].str[1]
    x['local_result_2'] = x['local_form'].str[2]
    x['local_result_3'] = x['local_form'].str[3]
    x['local_result_4'] = x['local_form'].str[4]

    #x['local_result_0'] = x['local_form'].str[0]
    x['visitor_result_1'] = x['visitor_form'].str[1]
    x['visitor_result_2'] = x['visitor_form'].str[2]
    x['visitor_result_3'] = x['visitor_form'].str[3]
    x['visitor_result_4'] = x['visitor_form'].str[4]

    # Les transformem en numerals (igual que result) 
    x = x.replace('l',0)
    x = x.replace('d',1)
    x = x.replace('w',2)

    #Eliminem les dades que no podrem reproduir
    x = x.drop('id', 1)
    x = x.drop('date', 1)
    x = x.drop('local_form', 1)
    x = x.drop('visitor_form', 1)
    x = x.drop('local_mark', 1)
    x = x.drop('visitor_mark', 1)
    x = x.drop('visitor_goals', 1)
    x = x.drop('local_goals', 1)

    x = x.drop('LALGR', 1)
    x = x.drop('LALGS', 1)
    x = x.drop('VAVGR', 1)
    x = x.drop('VAVGS', 1)

    x = x.fillna(0)

    
    return x


## Predicció

In [174]:
#Funció que predeix el resultat d'un partit (que carrega desde un csv)
#a partir del model que hem entrenat i exportat abans.
#El partit del csv ha de ser exactament igual que els que utilitzavem per entrenar el model, amb l'unica
#diferencia que no te el camp 'result' (que es el que volem predir)

def predict(x_file) :
    
    #prepara el partit a predir
    x = prepare_x(x_file)
    #carrega el model ja entrenat
    decision_tree_classifier = joblib.load('../TF_data/predictor_model.pkl')
    #i fa la predicció
    result =  decision_tree_classifier.predict(x)
    return result

print "Resultat: "
print predict("../TF_data/1-final-predictor.csv")

Resultat: 
[ 1.]
